In [1]:
import pandas as pd

# IMPORT DATA

In [2]:

df_bom = pd.read_excel('https://github.com/reaniumpy/python/blob/main/INPUT_MRP.xlsx?raw=true',sheet_name='BOM')
df_bom.head(2)

,parent,component,quantity per assembly
0,K1,THACHCAO-K1,1.0
1,K1,BIAGIAY-K1,1.0


In [3]:
df_item_master = pd.read_excel('https://github.com/reaniumpy/python/blob/main/INPUT_MRP.xlsx?raw=true',sheet_name='ITEM MASTER')
df_item_master['leadtime_days'] = df_item_master['leadtime_days'].astype('int')
df_item_master.head(2)

,item,type,leadtime_days
0,BIAGIAY-K1,RAW,2
1,BIAGIAY-K2,RAW,2


In [4]:
df_supply = pd.read_excel('https://github.com/reaniumpy/python/blob/main/INPUT_MRP.xlsx?raw=true',sheet_name='SUPPLY')
df_supply.head(3)

,item,2022-04-15 00:00:00,2022-04-16 00:00:00,2022-04-17 00:00:00,2022-04-18 00:00:00,2022-04-19 00:00:00
0,K1,13,9.0,NaN,NaN,NaN
1,K2,21,NaN,NaN,NaN,NaN
2,K3,18,3.0,1.0,NaN,NaN


In [5]:
df_forecast = pd.read_excel('https://github.com/reaniumpy/python/blob/main/INPUT_MRP.xlsx?raw=true',sheet_name='DEMAND')
df_forecast.head(3)

,item,2022-04-15 00:00:00,2022-04-16 00:00:00,2022-04-17 00:00:00,2022-04-18 00:00:00,2022-04-19 00:00:00
0,K1,29,4,2,19,21
1,K3,6,2,3,12,19
2,K2,39,16,5,17,23


# CALCULATE

### Bước 1: Tính số lượng net requirement (planned receipt) cho Thành phẩm (Finished Goods)

In [6]:
# Tính net requirement
df_net_pivot = df_forecast.set_index("item").subtract(
    df_supply.set_index("item"), axis=1, fill_value=0
)

df_net_pivot

,2022-04-15 00:00:00,2022-04-16 00:00:00,2022-04-17 00:00:00,2022-04-18 00:00:00,2022-04-19 00:00:00
item,,,,,
BIAGIAY-K1,-4.0,-10.0,NaN,NaN,NaN
BIAGIAY-K2,-3.0,-12.0,NaN,NaN,NaN
BIAGIAY-K3,-3.0,-13.0,NaN,NaN,NaN
K1,16.0,-5.0,2.0,19.0,21.0
K2,18.0,16.0,5.0,17.0,23.0
K3,-12.0,-1.0,2.0,12.0,19.0
THUNGCARTON,-5.0,NaN,NaN,NaN,NaN


In [7]:
# Lọc ra net req của hàng FG
item_fg = df_item_master.loc[df_item_master['type'] == 'FG','item'].unique()
fg_df_net_pivot = df_net_pivot.loc[item_fg].reset_index()
fg_df_net_pivot

,item,2022-04-15 00:00:00,2022-04-16 00:00:00,2022-04-17 00:00:00,2022-04-18 00:00:00,2022-04-19 00:00:00
0,K1,16.0,-5.0,2.0,19.0,21.0
1,K2,18.0,16.0,5.0,17.0,23.0
2,K3,-12.0,-1.0,2.0,12.0,19.0


In [8]:
# Unpivot net requirement của FG
fg_df_net_melt = fg_df_net_pivot.melt('item',var_name='period',value_name='FG planned receipt')

# Chuyển period thành kiểu string và sort theo item
fg_df_net_melt['period'] = fg_df_net_melt['period'].astype('str')
fg_df_net_melt = fg_df_net_melt.sort_values(by='item')
fg_df_net_melt

,item,period,FG planned receipt
0,K1,2022-04-15,16.0
3,K1,2022-04-16,-5.0
6,K1,2022-04-17,2.0
9,K1,2022-04-18,19.0
12,K1,2022-04-19,21.0
1,K2,2022-04-15,18.0
4,K2,2022-04-16,16.0
7,K2,2022-04-17,5.0
10,K2,2022-04-18,17.0
13,K2,2022-04-19,23.0


In [9]:
def cal_receipt(sr_in):
    # Khởi tạo với số lượng remain bằng 0
    total_remain = 0
    
    # Với từng index (i) , net value (x) của Series được đưa vào
    for i, x in sr_in.iteritems():
        # Nếu net value âm (stock dư) thì sẽ để dành cho việc phân phối qua period kế tiếp (khi x>0)
        if x < 0:
            # Lưu số dư vào biến remain
            total_remain = total_remain + x
            
            # Cho net value = 0 
            sr_in[i] = 0

        # Nếu net value dương (cần stock) 
        if x > 0 and total_remain < 0:
                
            # Nếu số stock cần nhiều hơn lượng dư thì khấu trừ tại chỗ
            if (x + total_remain) >= 0:
                sr_in[i] = sr_in[i] + total_remain
                total_remain = 0 

            # Nếu số stock cần ít hơn số stock dư thì chia làm 2 phần
            # 1 phần khấu trừ tại chỗ, 1 phần để lại cho period kế tiếp
            else:
                sr_in[i] = 0
                total_remain = total_remain + x
    return sr_in

# Tính planned receipt 
fg_df_net_melt['FG planned receipt'] = fg_df_net_melt.groupby('item')['FG planned receipt'].apply(cal_receipt)
fg_df_net_melt

<ipython-input-9-a174c81d6f98>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, x in sr_in.iteritems():
<ipython-input-9-a174c81d6f98>:31: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  fg_df_net_melt['FG planned receipt'] = fg_df_net_melt.groupby('item')['FG planned receipt'].apply(cal_receipt)


,item,period,FG planned receipt
0,K1,2022-04-15,16.0
3,K1,2022-04-16,0.0
6,K1,2022-04-17,0.0
9,K1,2022-04-18,16.0
12,K1,2022-04-19,21.0
1,K2,2022-04-15,18.0
4,K2,2022-04-16,16.0
7,K2,2022-04-17,5.0
10,K2,2022-04-18,17.0
13,K2,2022-04-19,23.0


### Bước 3: Tính số lượng hàng yêu cầu (raw GROSS requirement) cho Nguyên vật liệu (Raw materials)

#### 3.1 Lấy số lượng hàng GROSS từ tính toán FG ở bước 2

In [10]:
# Tính số lượng raw gross requirement cho raw
raw_df_bom = fg_df_net_melt.merge(df_bom,left_on='item',right_on='parent').drop(columns='parent')
raw_df_bom['RAW gross requirement'] = raw_df_bom['FG planned receipt'] * raw_df_bom['quantity per assembly']
raw_df_bom.head(2)

,item,period,FG planned receipt,component,quantity per assembly,RAW gross requirement
0,K1,2022-04-15,16.0,THACHCAO-K1,1.0,16.0
1,K1,2022-04-15,16.0,BIAGIAY-K1,1.0,16.0


In [11]:
# Tinh raw gross requirement cho từng raw materials
raw_df_gross = raw_df_bom[['component','period','RAW gross requirement']]
raw_df_gross = raw_df_gross.groupby(['component','period']).sum().reset_index()
raw_df_gross = raw_df_gross.rename(columns={'component':'item'})
raw_df_gross.head(6)

,item,period,RAW gross requirement
0,BIAGIAY-K1,2022-04-15,16.0
1,BIAGIAY-K1,2022-04-16,0.0
2,BIAGIAY-K1,2022-04-17,0.0
3,BIAGIAY-K1,2022-04-18,16.0
4,BIAGIAY-K1,2022-04-19,21.0
5,BIAGIAY-K2,2022-04-15,18.0


#### 3.2 Lấy số lượng hàng stock (on hand + in transit)

In [12]:
df_net_pivot

,2022-04-15 00:00:00,2022-04-16 00:00:00,2022-04-17 00:00:00,2022-04-18 00:00:00,2022-04-19 00:00:00
item,,,,,
BIAGIAY-K1,-4.0,-10.0,NaN,NaN,NaN
BIAGIAY-K2,-3.0,-12.0,NaN,NaN,NaN
BIAGIAY-K3,-3.0,-13.0,NaN,NaN,NaN
K1,16.0,-5.0,2.0,19.0,21.0
K2,18.0,16.0,5.0,17.0,23.0
K3,-12.0,-1.0,2.0,12.0,19.0
THUNGCARTON,-5.0,NaN,NaN,NaN,NaN


In [13]:
# Get stock (on hand + intransit) of Raw only
item_raw = df_item_master.loc[df_item_master['type'] == 'RAW','item'].unique()
raw_df_stock = df_net_pivot[df_net_pivot.index.isin(item_raw)]

# Unpivot
raw_df_stock_melt = (
    raw_df_stock.reset_index()
    .melt("item", var_name="period", value_name="RAW gross requirement")
    .sort_values(by="item")
).fillna(0)

raw_df_stock_melt['period'] = raw_df_stock_melt['period'].astype('str')
raw_df_stock_melt.head()

,item,period,RAW gross requirement
0,BIAGIAY-K1,2022-04-15,-4.0
16,BIAGIAY-K1,2022-04-19,0.0
4,BIAGIAY-K1,2022-04-16,-10.0
12,BIAGIAY-K1,2022-04-18,0.0
8,BIAGIAY-K1,2022-04-17,0.0


#### 3.3 Ghép gross và stock

In [14]:
raw_df_net = pd.concat([raw_df_gross,raw_df_stock_melt]).groupby(['item','period']).sum().reset_index()

#### 3.2 Tính lượng hàng net (planned receipt) bằng cách lấy gross - stock

In [15]:
# Lấy gross trừ stock, nếu dư phân phối qua kì tiếp theo
raw_df_net ['RAW planned receipt'] = raw_df_net.groupby('item')['RAW gross requirement'].apply(cal_receipt)
raw_df_net = raw_df_net.drop(columns='RAW gross requirement')
raw_df_net.head(5)


<ipython-input-9-a174c81d6f98>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, x in sr_in.iteritems():
<ipython-input-15-f9d061443bf4>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  raw_df_net ['RAW planned receipt'] = raw_df_net.groupby('item')['RAW gross requirement'].apply(cal_receipt)


,item,period,RAW planned receipt
0,BIAGIAY-K1,2022-04-15,12.0
1,BIAGIAY-K1,2022-04-16,0.0
2,BIAGIAY-K1,2022-04-17,0.0
3,BIAGIAY-K1,2022-04-18,6.0
4,BIAGIAY-K1,2022-04-19,21.0


In [16]:
raw_df_net['release'] = raw_df_net['RAW planned receipt'].shift(-1)
raw_df_net.head(5)

,item,period,RAW planned receipt,release
0,BIAGIAY-K1,2022-04-15,12.0,0.0
1,BIAGIAY-K1,2022-04-16,0.0,0.0
2,BIAGIAY-K1,2022-04-17,0.0,6.0
3,BIAGIAY-K1,2022-04-18,6.0,21.0
4,BIAGIAY-K1,2022-04-19,21.0,15.0


### BƯỚC 4: Tính planned order release của RAW

In [17]:
# Tạo dictionary chứa leadtime của từng item
dict_lead_time = dict(zip(df_item_master["item"], df_item_master["leadtime_days"]))

In [18]:
dict_lead_time

{'BIAGIAY-K1': 2,
 'BIAGIAY-K2': 2,
 'BIAGIAY-K3': 2,
 'THACHCAO-K1': 2,
 'THACHCAO-K2': 2,
 'THACHCAO-K3': 2,
 'KHAYNHUA': 2,
 'COTOMAU': 2,
 'DAYNHUA': 2,
 'MAUNUOC': 2,
 'THUNGCARTON': 2,
 'K1': 0,
 'K2': 0,
 'K3': 0}

In [21]:
dict_lead_time['K1']

0

In [ ]:
raw_df_net['planned order release'] = raw_df_net.groupby("item")["RAW planned receipt"].apply(
    lambda x: x.shift(-dict_lead_time[x.name])
)
raw_df_net.head()

In [ ]:
def nhan_doi(x):
    return x* 2
nhan_doi(3)

In [ ]:
nhan_doi_lam = lambda x: x* 2
nhan_doi_lam (3)

In [ ]:
# def ham_tu_dn(x): # x là Series đầu vào
#     x = x.shift(-dict_lead_time[x.name])
#     return x

# lambda x: x.shift(-dict_lead_time[x.name])

In [ ]:
raw_df_net['planned order release'] = raw_df_net.groupby("item")["RAW planned receipt"].apply(
    ham_tu_dn
)
raw_df_net.head()

In [ ]:
# Tách ra các đơn hàng cần order ngay tức khắc 
raw_df_order_now = raw_df_net.groupby("item")["RAW planned receipt"].apply(
    lambda x: x[: dict_lead_time[x.name]].sum()
).to_frame().reset_index()

raw_df_order_now.rename(columns={'RAW planned receipt':'planned order release'},inplace=True)

raw_df_order_now['period'] =  '2022-04-14_today'
raw_df_order_now


In [ ]:
df_concat = pd.concat([raw_df_net,raw_df_order_now])
df_concat.head()

In [ ]:
df_concat.sort_values(by=['item','period']).head(10)

In [ ]:
df_final = df_concat[['item','period','RAW planned receipt','planned order release']]
df_final_2 = df_final.pivot_table(index='item',columns='period',aggfunc='sum',values='planned order release').astype('int')

In [ ]:
df_final_2.to_excel('OUTPUT.xlsx')

In [ ]:
df_final_2